In [5]:
import selenium.webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException

import time
import atexit
import os
import pandas as pd
from glob import glob

import re
from urllib import parse

In [18]:
#전역 변수 선언
TIMEOUT = 10
ROOT_DIR = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
DOWNLOAD_DIR = f'{ROOT_DIR}\\download\\tta'
print(DOWNLOAD_DIR)

#사이트 주소
URL = 'http://pms.visitincheon.or.kr:9090/pms/index.php?act=dispMemberLoginForm'

c:\development\workspaces\python\weekend\AZPower\Upgrade_210110\download\tta


In [7]:
#chrome webdriver를 생성한다.
# - download_path: 파일을 다운로드 받을 위치
# - headless: True이면 브라우저 화면 없이 background로 처리
def create_chrome(download_path, headless=False) :
    options = ChromeOptions()
    if headless:
        options.add_argument('headless')
        options.add_argument('--disable-gpu')
    options.add_experimental_option('prefs', {
        'download.default_directory' : download_path,
        'download.prompt_for_download' : False
    })

    chrome = Chrome(chrome_options = options)

    if headless :
        _enable_download_on_chrome_headless(chrome, download_path)

    #프로세스 종료시 크롬을 종료시키는 로직
    atexit.register(_close_chrome(chrome))
    
    return chrome

#headless=True 일때 다운로드 안되는(크롬 보안문제) 문제 해결
def _enable_download_on_chrome_headless(browser, download_dir) :
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')

    params = {
        'cmd': 'Page.setDownloadBehavior',
        'params': {
            'behavior': 'allow',
            'downloadPath': download_dir
        }
    }
    browser.execute("send_command", params)

#크롬 종료~
def _close_chrome(chrome) :
    def close() :
        chrome.close()
    return close

def chrome_wait(wait, condition):
    try:
        element = wait.until(condition)
        return element
    except TimeoutException:
        print(f"[DEBUG] 페이지 로딩 시간 초과")

In [25]:
ID = "tkrhkrkantu"
PW = "mice1130!@"

In [94]:
#크롬 브라우저 생성 및 첫번째 카테고리를 선택
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)
browser.get(URL)

In [95]:
idEle = browser.find_element_by_xpath('//*[@id="account-signup"]/fieldset/input[@name="user_id"]')
idEle.clear()
idEle.send_keys(ID)

pwEle = browser.find_element_by_xpath('//*[@id="account-signup"]/fieldset/input[@name="password"]')
pwEle.clear()
pwEle.send_keys(PW)

loginEle = browser.find_element_by_xpath('//*[@id="account-signup"]/fieldset/input[@type="submit"]')
loginEle.click()

In [96]:
targetEle = browser.find_element_by_xpath('//*[@id="gnb"]/ul/li/a[text()="(신)지원"]')
targetEle.click()

In [101]:
data = []

for page in range(1, 43) :

    if page != 1 :
        browser.find_element_by_xpath(f'//div[@class="pagination"]//a[text()="{page}]"]').click()
        time.sleep(0.3)

    trEles = browser.find_elements_by_xpath('//*[@id="board_list"]/table/tbody/tr')
    trCnt = len(trEles)
    for index in range(trCnt) :
        xpath = f'//*[@id="board_list"]/table/tbody/tr[{index + 1}]'

        num = browser.find_element_by_xpath(xpath + "/td[1]").text
        status = browser.find_element_by_xpath(xpath + "/td[2]").text
        subject = browser.find_element_by_xpath(xpath + "/td[3]/a[1]").text
        writer = browser.find_element_by_xpath(xpath + "/td[4]").text
        
        link = browser.find_element_by_xpath(xpath + "/td[3]/a[1]")
        link.click()

        print(num, status, subject, writer)
        
        reg_date = browser.find_element_by_xpath('//*[@id="content"]//span[@class="time"]').text
        
        feedback_date = ''
        comment = ''
        try :    
            feedback_date = browser.find_element_by_xpath('//li[contains(@id, "comment_")][last()]/div[1]/p').text
            comment = browser.find_element_by_xpath('//li[contains(@id, "comment_")][last()]/div[2]/p').text
        except NoSuchElementException:
            feedback_date = ''
            comment = ''
        data.append([num, status, subject, reg_date, feedback_date, comment])

        browser.save_screenshot(f"./images/{num}_{status}.png")

        print(num)

        browser.execute_script("window.history.go(-1)")
        time.sleep(0.3)





823 요청  [컨벤션마케팅팀] 배정판 기능 추가 요청 최보아


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]//span[@class="time"]"}
  (Session info: chrome=87.0.4280.141)
